<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_forecasting/blob/main/Vector_AR_%2B_SLR_with_average_solar_output_Germany.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

path = '/content/drive/MyDrive/Master_thesis/Datasets/Merged(GER+weather).csv'

df = read_csv(path, header =0, index_col=0) 
df.head()
df.columns

Index(['cet_cest_timestamp', 'DE_KN_industrial1_pv_1',
       'DE_KN_industrial1_pv_2', 'DE_KN_industrial2_pv',
       'DE_KN_industrial3_pv_facade', 'DE_KN_industrial3_pv_roof',
       'DE_KN_residential1_pv', 'DE_KN_residential3_pv',
       'DE_KN_residential4_pv', 'DE_KN_residential6_pv', 'Berlin_time', 'Year',
       'Month', 'Season', 'Week', 'Day', 'Year_day', 'Hour', 'local_time',
       'radiation_surface'],
      dtype='object')

In [ ]:

drop_cols = [i for i in range(10,19)]
drop_cols.append(0)
df = df.fillna(0)

#df =df.drop(df.columns[drop_cols],axis=1)
df.head()

df.columns


Index(['DE_KN_industrial1_pv_1', 'DE_KN_industrial1_pv_2',
       'DE_KN_industrial2_pv', 'DE_KN_industrial3_pv_facade',
       'DE_KN_industrial3_pv_roof', 'DE_KN_residential1_pv',
       'DE_KN_residential3_pv', 'DE_KN_residential4_pv',
       'DE_KN_residential6_pv', 'radiation_surface'],
      dtype='object')

In [ ]:
pv_col = [col for col in df.columns if "pv" in col]

df["pv_avg"] = df[pv_col].mean(axis=1, skipna=False)
df.head()



,DE_KN_industrial1_pv_1,DE_KN_industrial1_pv_2,DE_KN_industrial2_pv,DE_KN_industrial3_pv_facade,DE_KN_industrial3_pv_roof,DE_KN_residential1_pv,DE_KN_residential3_pv,DE_KN_residential4_pv,DE_KN_residential6_pv,radiation_surface,pv_avg
utc_timestamp,,,,,,,,,,,
2016-02-01 05:00:00+00:00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000000
2016-02-01 06:00:00+00:00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.467,0.000000
2016-02-01 07:00:00+00:00,0.00,0.000,0.692,0.067,0.007,0.291,0.0,0.367,0.000,31.309,0.158222
2016-02-01 08:00:00+00:00,0.02,0.000,2.712,0.813,0.063,1.258,0.0,1.628,0.038,112.503,0.725778
2016-02-01 09:00:00+00:00,0.32,0.195,3.113,3.284,0.135,1.677,0.0,1.813,0.607,208.440,1.238222


In [ ]:
df2 = df[['pv_avg','radiation_surface']]
df2

,pv_avg,radiation_surface
utc_timestamp,,
2016-02-01 05:00:00+00:00,0.000000,0.000
2016-02-01 06:00:00+00:00,0.000000,0.467
2016-02-01 07:00:00+00:00,0.158222,31.309
2016-02-01 08:00:00+00:00,0.725778,112.503
2016-02-01 09:00:00+00:00,1.238222,208.440
...,...,...
2018-01-31 15:00:00+00:00,0.061222,51.763
2018-01-31 16:00:00+00:00,0.001111,0.060
2018-01-31 17:00:00+00:00,0.000000,0.000


In [ ]:
pip install statsmodels --upgrade

In [ ]:
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = df2[df2.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(df.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

In [ ]:
pip install statsmodels --upgrade

In [ ]:
df2.head()

,pv_avg,radiation_surface
utc_timestamp,,
2016-02-01 05:00:00+00:00,0.000000,0.000
2016-02-01 06:00:00+00:00,0.000000,0.467
2016-02-01 07:00:00+00:00,0.158222,31.309
2016-02-01 08:00:00+00:00,0.725778,112.503
2016-02-01 09:00:00+00:00,1.238222,208.440


In [ ]:
#creating the train and validation set
train = df2[:int(0.8*(len(df2)))]
valid = df2[int(0.8*(len(df2))):]


#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.var_model import forecast

model = VAR(endog=train)
model_fit = model.fit()

model_fit.params
#print(model_fit)

# make prediction on validation
prediction = model_fit.forecast(model.y, steps=len(valid))

for_solar = prediction[:,0]
print(len(for_solar))
#plt.plot(for_solar)

2193


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:595: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0


In [ ]:
#converting predictions to dataframe

from sklearn.metrics import mean_squared_error

pred = pd.DataFrame(index=range(0,len(prediction)),columns=df2.columns)
for j in range(0,2):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in df2.columns:
    print('mse value for', i, 'is : ', mean_squared_error(pred[i], valid[i]))

mse value for pv_avg is :  1.5555488039461862
mse value for radiation_surface is :  48571.18061973803


In [ ]:
#make final predictions
model = VAR(endog=df2)
model_fit = model.fit()
yhat = model_fit.forecast(model.y, steps=1)
print(yhat)


[[-2.15771802e-03  1.57120838e+01]]


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:595: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  after removing the cwd from sys.path.


#Simple linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df2["radiation_surface"]
y = df2["pv_avg"]

print("Shape of X", X.shape)
print("Shape of y", y.shape)

X_reshaped = X.values.reshape(-1,1) 
print("Shape of X", X_reshaped.shape) 

y_reshaped = y.values.reshape(-1,1)
print("Shape of y", y_reshaped.shape)


X_train, X_test, y_train, y_test = train_test_split( X_reshaped, y_reshaped, test_size=0.3, shuffle = False)

Shape of X (10965,)
Shape of y (10965,)
Shape of X (10965, 1)
Shape of y (10965, 1)


In [ ]:
from sklearn import metrics
reg = LinearRegression().fit(X_train, y_train)
print("R^2 of Linear Regression on the train set =", reg.score(X_train, y_train))
print("R^2 of Linear Regression on the test set =", reg.score(X_test, y_test))

y_pred = reg.predict(X_test)
print("MAE:", metrics.mean_absolute_error(y_test,y_pred))
print("MSE:", metrics.mean_squared_error(y_test,y_pred))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

R^2 of Linear Regression on the train set = 0.6110351779778092
R^2 of Linear Regression on the test set = -0.7909478608507283
MAE: 0.6781532546808299
MSE: 1.0418488291732313
RMSE: 1.0207099632967396
